In [1]:
import numpy as np
import rawpy
import cv2
import matplotlib.pyplot as plt
from skimage import img_as_ubyte

In [12]:
def basic_showImg(img, size=4):
    '''Shows an image in a numpy.array type. Syntax:
        basic_showImg(img, size=4), where
            img = image numpy.array;
            size = the size to show the image. Its value is 4 by default.
    '''
    plt.figure(figsize=(size,size))
    #img = np.rot90(img)
    plt.imshow(img)
    plt.show()


def percentile_whitebalance(image, percentile_value):
        whitebalanced = img_as_ubyte(
                (image*1.0 / np.percentile(image, 
                percentile_value, axis=(0, 1))).clip(0, 1))
        return whitebalanced

def toOpenCVU8(img):
    out = img * 255
    out[out < 0] = 0
    out[out > 255] = 255
    out = cv2.cvtColor(out.astype(np.uint8), cv2.COLOR_RGB2BGR)
    return out

def imsave(img, title='img', dst=f'../test_images/test.jpg'):
    size = 1500
    #img = np.rot90(img)
    if max(img.shape[1], img.shape[0]) >= size:
        if img.shape[1] >= img.shape[0]:
            width = size
            height = int(img.shape[0] * size / img.shape[1])
        elif img.shape[0] >= img.shape[1]:
            width = int(img.shape[1] * size / img.shape[0])
            height = size
        dim = (width, height)
        img = cv2.resize(img, dim)
    cv2.imwrite(str(dst), img)

def get_target_colour_matrix(targets):
    list_of_targets = []
    for target in targets:
        list_of_targets.append(targets[target])
    target_matrix = np.vstack(list_of_targets)
    return target_matrix

def get_avg_colour_matrix(avgs):
    list_of_avgs = []
    for avg in avgs:
        list_of_avgs.append(avgs[avg])
    avg_matrix = np.vstack(list_of_avgs)
    return avg_matrix

In [3]:
lower_blue = np.array([135, 65, 35])
upper_blue = np.array([165, 255, 255])
lower_green = np.array([40, 52, 70])
upper_green = np.array([82, 255, 255]) 
lower_yellow = np.array([20, 100, 100])
upper_yellow = np.array([30, 255, 255])

lower_red = np.array([0, 50, 50])
upper_red = np.array([10, 255, 255])

lower_black = np.array([0, 0, 0])
upper_black = np.array([179, 255, 75])
lower_white = np.array([0, 0, 180]) 
upper_white = np.array([0, 0, 255]) 

COLOURS = ('blue', 'green', 'yellow', 'red', 'black', 'white')

COLOUR_RANGE = {
    'blue': (lower_blue, upper_blue),
    'green': (lower_green, upper_green),
    'yellow': (lower_yellow, upper_yellow),
    'red': (lower_red, upper_red),
    'black': (lower_black, upper_black),
    'white': (lower_white, upper_white)
}

TARGETS = {
    'blue': np.array([26,0,165]),
    'green': np.array([30,187,22]),
    'yellow': np.array([252,220,10]),
    'red': np.array([240,0,22]),
    'black': np.array([0,0,0]),
    'white': np.array([255,255,255])
}

TARGETS_NORM = {
    'blue': np.array([26,0,165]) / 255.0,
    'green': np.array([30,187,22]) / 255.0,
    'yellow': np.array([252,220,10]) / 255.0,
    'red': np.array([240,0,22]) / 255.0,
    'black': np.array([0,0,0]) / 255.0,
    'white': np.array([255,255,255]) / 255.0
}

AVG = { # to be extracted from the images
    'blue': np.array([0.0, 0.0, 0.0]),
    'green': np.array([0.0, 0.0, 0.0]),
    'yellow': np.array([0.0, 0.0, 0.0]),
    'red': np.array([0.0, 0.0, 0.0]),
    'black': np.array([0.0, 0.0, 0.0]),
    'white': np.array([0.0, 0.0, 0.0])
}

kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 10))

In [18]:
file_range = range(9,17)  # to be specified: the range of files to be processed
in_path = ''
out_path = ''
for i in file_range:
    in_path = f'../test_images/test2/original/{i}.CR3' # to be specified: the path of the file to be processed
    out_path = f'../test_images/test2/processed/{i}.jpg' # to be specified: the path of the file to be saved

    image = rawpy.imread(in_path)
    rgb = image.postprocess().astype(np.uint8)
    #print(f'{i} read successfully')
    whitebalanced = percentile_whitebalance(rgb, 97.5)

    hsv_image = cv2.cvtColor(whitebalanced, cv2.COLOR_RGB2HSV)
    for colour in COLOURS:
        mask = cv2.inRange(hsv_image, COLOUR_RANGE[colour][0], COLOUR_RANGE[colour][1])
        mask_updated = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
        contours, _ = cv2.findContours(mask_updated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        contours = list(filter(lambda x: len(cv2.approxPolyDP(x, 0.01 * cv2.arcLength(x, True), True)) == 4, contours))
        contours = sorted(contours, key=cv2.contourArea, reverse=True)

        if len(contours) > 0:
            bounding_rect = cv2.boundingRect(contours[0])
            roi = rgb[bounding_rect[1]:bounding_rect[1] + bounding_rect[3], 
                    bounding_rect[0]:bounding_rect[0] + bounding_rect[2]]
            
        avg = np.mean(roi, axis=(0, 1))
        #print(f'{i} {colour} average: {avg}')
        AVG[colour] = avg

    target_colour_matrix = get_target_colour_matrix(TARGETS_NORM)

    #print(f'{i} target colour matrix: {target_colour_matrix}')

    average_colour_matrix = get_avg_colour_matrix(AVG)

    #print(f'{i} average colour matrix: {average_colour_matrix}')

    U, s, Vt = np.linalg.svd(average_colour_matrix.T @ target_colour_matrix)
    color_correction_matrix = U @ Vt
    corrected_image_svd = cv2.transform(rgb, color_correction_matrix)

    to_save = cv2.cvtColor(corrected_image_svd, cv2.COLOR_RGB2BGR)
    to_save_whitebalanced = percentile_whitebalance(to_save, 97.5)
    imsave(to_save_whitebalanced, dst=out_path)
    # print(f'{i} saved successfully')
    

9 saved successfully
10 saved successfully
11 saved successfully
12 saved successfully
13 saved successfully
14 saved successfully
15 saved successfully
16 saved successfully
